# Library

In [1]:
!pip install transformers
!pip install Sastrawi
!pip install langdetect

In [2]:
# import library untuk memuat dan menganalisis data
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

# import library untuk text cleaning
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re

# import library untuk deteksi bahasa
from langdetect import detect, LangDetectException

# import pipeline
from sklearn.pipeline import Pipeline

# import library untuk membuat model
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import pipeline
from tqdm import tqdm
import torch
import os

# import library untuk menyimpan model yang sudah dibuat
import joblib 

# Memuat Data

In [3]:
Data = "https://github.com/keaganwjy/datathon/raw/refs/heads/main/reviews_genshin_impact_raw.csv"
jumlah_baris = 2000 # jumlah baris yang akan dimasukan.as_integer_ratio
df_raw = pd.read_csv(Data) # membaca CSV
df = df_raw.head(jumlah_baris).copy() # menggunakan .head() untuk mengambil baris teratas secara langsung
df.head()

,Nama Pengguna,Teks Ulasan,Rating Bintang,Tanggal Ulasan,Jumlah Likes
0,Pengguna Google,pelit pengen karakter aja susah malah dapet yg...,1,2025-06-20 14:40:33,0
1,Pengguna Google,farming artefak engga pernah dikasih stat/sub ...,1,2025-06-20 14:00:33,0
2,Pengguna Google,"udah ngga worth it, perbaiki diri mu genshin, ...",1,2025-06-20 13:49:19,0
3,Pengguna Google,kasih fitur skip.,1,2025-06-20 13:22:21,0
4,Pengguna Google,peak update,5,2025-06-20 13:16:04,0


# Data Cleaning Dengan Pipeline

1. Penyesuaian Data
2. Pembersihan Text
3. Deteksi Bahasa

In [4]:
# class untuk menyesuaikan tipe data
class PenyesuaianTipeData(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # mengubah 'Tanggal Ulasan' menjadi format datetime
        # ini memungkinkan kita untuk melakukan analisis berbasis waktu nanti
        X['Tanggal Ulasan'] = pd.to_datetime(X['Tanggal Ulasan'])
        
        # memastikan 'Rating Bintang' dan 'Jumlah Likes' adalah angka (integer)
        # error='coerce' akan mengubah nilai yang tidak bisa diubah menjadi angka (misal: teks) menjadi NaN (kosong)
        X['Rating Bintang'] = pd.to_numeric(X['Rating Bintang'], errors='coerce')
        X['Jumlah Likes'] = pd.to_numeric(X['Jumlah Likes'], errors='coerce')
        
        # kita bisa membuang baris yang rating atau likes-nya menjadi NaN, atau mengisinya dengan 0
        X.dropna(subset=['Rating Bintang'], inplace=True) # rating wajib ada
        # kode baru (praktik terbaik dan aman)
        X['Jumlah Likes'] = X['Jumlah Likes'].fillna(0)
        
        # mengubah tipe data menjadi integer untuk efisiensi memori
        X['Rating Bintang'] = X['Rating Bintang'].astype(int)
        X['Jumlah Likes'] = X['Jumlah Likes'].astype(int)
        
        return X

In [5]:
# class untuk pembersihan teks
class PembersihanText(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # inisialisasi stemmer dan stopword remover
        factory_stemmer = StemmerFactory()
        stemmer = factory_stemmer.create_stemmer()
        factory_stopword = StopWordRemoverFactory()
        stopword_remover = factory_stopword.create_stop_word_remover()
        
        # buat kamus slang yang lebih lengkap (ini bisa terus Anda kembangkan)
        slang_dict = {
        'bgt': 'banget', 'gak': 'tidak', 'ga': 'tidak', 'kalo': 'kalau', 'gacha': 'gacha',
        'dgn': 'dengan', 'krn': 'karena', 'yg': 'yang', 'utk': 'untuk', 'mantap': 'mantap',
        'keren': 'keren', 'bug': 'bug', 'ngebug': 'bug', 'loding': 'loading', 'ngelek': 'lag',
        'ngeframe': 'frame', 'drop': 'drop', 'jelek': 'jelek', 'bangettt': 'banget',
        'sih': '', 'nya': '', 'aja': 'saja', 'kok': '', 'sih': ''
        }
        
        def preprocess_text(text):
            # pastikan input adalah string
            if not isinstance(text, str):
                return ""
            # 1. case folding
            text = text.lower()
            # 2. hapus noise (URL, mention, hashtag, karakter non-alfabet)
            text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
            text = re.sub(r'\@\w+|\#', '', text)
            text = re.sub(r'[^a-z\s]', '', text)
            # 3. normalisasi kata slang
            words = text.split()
            normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
            text = " ".join(normalized_words)
            # 4. Stopword Removal
            text = stopword_remover.remove(text)
            # 5. stemming
            text = stemmer.stem(text)
            return text
        
        # menggunakan .copy() untuk menghindari SettingWithCopyWarning
        X_processed = X.copy()
        X_processed['Ulasan Bersih'] = X_processed['Teks Ulasan'].apply(preprocess_text)
        return X_processed
                
        

        

In [6]:
# class untuk  medeteksi bahasa 
class DeteksiBahasa(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        kolom_krusial = ['Ulasan Bersih', 'Rating Bintang', 'Tanggal Ulasan']
        X.dropna(subset=kolom_krusial, inplace=True)
        
        # cek jumlah baris setelah pembersihan untuk melihat perbedaannya
        X.reset_index(drop=True, inplace=True)
        
        # 1. Definisikan fungsi yang aman (pastikan nama ini yanga digunakan)
        def deteksi_bahasa_aman(teks):
            # cek apakah inputnya bukan string (misalnya NaN)
            if pd.isna(teks) or not isinstance(teks, str):
                return 'undefined'
            
            # cek apakah teks terlalu pendek
            if len(teks.strip()) < 10:
                return 'pendek'
            
            # coba deteksi bahasa
            try:
                return detect(teks)
            except LangDetectException:
                return 'error'
        
    # 2. Terapkan fungsi yang benar pada dataframe anda
    # asumsikan 'X' adalah dataframe anda
    # perbaikan di sini: Panggil 'deteksi_bahasa_aman'
        X['bahasa'] = X['Ulasan Bersih'].apply(deteksi_bahasa_aman)
        
        # hasil
        print("\nDistribusi Bahasa pada Dataset")
        print(X['bahasa'].value_counts())
        return X
    
    

# Labelling

In [19]:
class Labelling(BaseEstimator, TransformerMixin):
            
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
        os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "60"  # Timeout jadi 60 detik
        classifier = pipeline(
            "zero-shot-classification",
            model="cahya/distilbert-base-indonesian",
            device=0  # atau -1 jika tidak ada GPU
        )
        
        print("Model berhasil dimuat.")

        # --- Definisikan Fungsi Pelabelan ---
        # Daftar kategori final Anda
        candidate_labels = ['Cerita', 'Gameplay', 'Grafis', 'Bugs & Error', 'Optimalisasi', 'Monetisasi & Gacha', 'Komunitas'] # Tambah/ubah sesuai kebutuhan

        def get_auto_labels(text):
            # Ambang batas skor agar sebuah label dianggap relevan
            threshold = 0.60 # Anda bisa menyesuaikan ini nanti

            try:
                result = classifier(text, candidate_labels, multi_label=True)

                # Ambil label yang skornya di atas ambang batas
                labels = [label for label, score in zip(result['labels'], result['scores']) if score > threshold]

                # Jika tidak ada label di atas ambang batas, kembalikan list kosong
                return labels if labels else []
            except Exception as e:
                # Jika terjadi error pada teks tertentu
                print(f"Error pada teks: {text[:50]}... | Error: {e}")
                return ["error"]

        # --- Terapkan pada Sampel Kecil untuk Uji Coba ---
        # PENTING: Jalankan pada sampel kecil dulu (misal: 1000-2000 baris)
        # untuk menghemat waktu dan memvalidasi proses.
        X_sample = X.head(2000).copy()
        X_sample = X_sample[X_sample['Ulasan Bersih'].notna() & (X_sample['Ulasan Bersih'].str.strip() != "")]

        X_sample['auto_labels'] = [
            classifier(text, candidate_labels, multi_label=True)
            for text in tqdm(X_sample['Ulasan Bersih'])
        ]
        print("Pelabelan otomatis selesai.")

        # Mari kita lihat hasilnya. Hasilnya akan sedikit berbeda, berupa dictionary.
        # Kita akan ekstrak labelnya saja.
        def extract_labels_from_result(result, threshold=0.60):
            return [label for label, score in zip(result['labels'], result['scores']) if score > threshold]

        X_sample['auto_labels_list'] = [extract_labels_from_result(res) for res in X_sample['auto_labels']]


        # Tampilkan kolom-kolom yang relevan
        print("\nContoh Hasil Pelabelan Otomatis:")
        print(X_sample[['Ulasan Bersih', 'auto_labels_list']].head(10))
        
        return X_sample

# Model

In [42]:
class Modelling(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Gunakan GPU jika tersedia
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load IndoBERT
        tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
        model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1").to(device)
        model.eval()

        # buat label sentimen manual (bisa juga diganti dengan anotasi jika tersedia)
        def simple_sentiment(text):
            text = text.lower()
            if any(k in text for k in ['jelek', 'buruk', 'crash', 'lag', 'sampah']):
                return "negatif"
            elif any(k in text for k in ['bagus', 'keren', 'mantap', 'seru']):
                return "positif"
            else:
                return "netral"
            
        # terapkan label sentimen
        X['sentimen'] = X['Ulasan Bersih'].apply(simple_sentiment)
        
        # encode label sentimen
        label_encoder = LabelEncoder()
        X['label_sentimen'] = label_encoder.fit_transform(X['sentimen']) # 0 = negatif, 1 = netral, 2 = positif
        
        # melakukan feature extractor untuk IndoBERT
        def get_bert_embedding(text, max_len=128):
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=max_len)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = model(**inputs)
            # Ambil CLS token sebagai representasi kalimat
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
            return cls_embedding
        
        # ambil embedding untuk seluruh data
        print("Mengekstrak embedding IndoBERT...")
        embeddings = np.array([get_bert_embedding(text) for text in tqdm(X['Ulasan Bersih'])])
        
        # latih model machine learning
        X_mod = embeddings
        y_mod = label_encoder.fit_transform(X['sentimen']) 
        
        X_train_mod, X_test_mod, y_train_mod, y_test_mod = train_test_split(X_mod, y_mod, test_size=0.2, random_state = 42)
        
        # latih model dengan Logistic Regression 
        clf = LogisticRegression(max_iter=1000)
        clf.fit(X_train_mod, y_train_mod)
        
        y_pred = clf.predict(X_test_mod)
        
        # Evaluasi
        print("\nHasil evaluasi model sentimen")
        print(classification_report(y_test_mod, y_pred, target_names=label_encoder.classes_))
        
        # prediksi dan kategorisasi game
        # terapkan prediksi ke seluruh data
        X['pred_sentimen'] = label_encoder.inverse_transform(clf.predict(X_mod))
        
        # Buat ringkasan per kategori berdasarkan label zero-shot sebelumnya
        from collections import Counter
        
        def ringkasan_kategori(X):
            hasil = []
            candidate = ['Cerita', 'Gameplay', 'Grafis', 'Bugs & Error', 
                        'Optimalisasi', 'Monetisasi & Gacha', 'Komunitas']
            for kategori in candidate:
                subset = X[X['auto_labels_list'].apply(lambda x: kategori in x)]
                total = len(subset)
                if total == 0:
                    continue
                counter = Counter(subset['pred_sentimen'])
                ringkas = {
                    'Kategori': kategori,
                    'Total Ulasan': total,
                    'Positif': counter['positif'],
                    'Negatif': counter['negatif'],
                    'Netral': counter['netral'],
                }
                hasil.append(ringkas)
            return pd.DataFrame(hasil)
        
        df_summary = ringkasan_kategori(X)
        print("\nRingkasan Sentimen per Kategori:")
        print(df_summary)

        # prediksi tingkat akurasi model
        accuracy = accuracy_score(y_test_mod,y_pred)
        print(f"\nTingkat akurasi model: {accuracy}")
        
        return X

# Execute

In [43]:
Pipe = Pipeline([
    ("Penyesuaian",PenyesuaianTipeData()),
    ("Pembersihan",PembersihanText()),
    ("Deteksi",DeteksiBahasa()),
    ("Labelling",Labelling()),
    ("Model", Modelling())
])

In [44]:
model = Pipe.transform(df)


Distribusi Bahasa pada Dataset
bahasa
id        1182
pendek     308
tl         147
en         104
lt          43
no          21
so          19
sw          18
sq          16
nl          14
da          14
sl          13
et          12
af          11
ca          10
ro          10
hr           9
tr           9
fi           8
it           8
cy           7
es           3
fr           3
hu           2
de           2
sk           2
pt           2
sv           1
pl           1
lv           1
Name: count, dtype: int64


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at cahya/distilbert-base-indonesian and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Model berhasil dimuat.


100%|██████████| 1979/1979 [05:08<00:00,  6.42it/s]


Pelabelan otomatis selesai.

Contoh Hasil Pelabelan Otomatis:
                                       Ulasan Bersih auto_labels_list
0          pelit ken karakter susah malah dapet lain               []
1  farming artefak engga pernah kasih statsub sta...               []
2  udah ngga worth it baik diri mu genshin dengar...               []
3                                   kasih fitur skip               []
4                                        peak update               []
5                                               seru               []
6  game ada kembang sama sekali scene kaku desain...               []
7  kuota ku habis cuman mendondlod data beri pili...               []
8                               bagus banget gamenya               []
9      game e apik poll lek hp ne kentang ojok maksa               []
Mengekstrak embedding IndoBERT...


100%|██████████| 1979/1979 [02:32<00:00, 13.00it/s]



Hasil evaluasi model sentimen
              precision    recall  f1-score   support

     negatif       0.54      0.32      0.40        22
      netral       0.94      0.93      0.93       246
     positif       0.85      0.93      0.89       128

    accuracy                           0.89       396
   macro avg       0.78      0.72      0.74       396
weighted avg       0.89      0.89      0.89       396


Ringkasan Sentimen per Kategori:
Empty DataFrame
Columns: []
Index: []

Tingkat akurasi model: 0.8939393939393939
